<a href="https://colab.research.google.com/github/UN-GCPDS/Curso-Corto-LLMs/blob/main/2.Entrenamiento_Tabnet/Aguas_Abajo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Logo UNAL CHEC](https://github.com/UN-GCPDS/curso_IA_CHEC/blob/main/logo_unal_chec.jpg?raw=1)

# **Entrenamiento modelo Tabnet**

## **Descripción**


### **Profesor - Sesión 1:** Andrés Marino Álvarez Meza y Diego Armando Pérez Rosero

In [1]:
%%capture
!pip install --upgrade shapely geopandas
import os
import math
import time
import random
import numpy as np
import pandas as pd
import networkx as nx
import geopandas as gpd
from scipy.spatial import ConvexHull
from shapely import LineString, Point
import zipfile
import os
!pip install -q gdown

In [2]:
import warnings
!gdown --id 1o_fZIhk6ErrtrM3eVZPF9s2qj8l4FoqS -O SuperEventos_Criticidad_AguasAbajo_CODEs.zip
!gdown --id 16VIuHLgPGpX4J723Wd48UAPhHivLuUaH -O Data_CHEC.zip

zip_path = "SuperEventos_Criticidad_AguasAbajo_CODEs.zip"
extract_dir = "CHEC"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)


zip_path = "Data_CHEC.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
# Ignorar los warnings específicos
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1o_fZIhk6ErrtrM3eVZPF9s2qj8l4FoqS
From (redirected): https://drive.google.com/uc?id=1o_fZIhk6ErrtrM3eVZPF9s2qj8l4FoqS&confirm=t&uuid=c1654165-d777-41b8-9a2f-4003392833a0
To: /content/SuperEventos_Criticidad_AguasAbajo_CODEs.zip
100% 214M/214M [00:01<00:00, 121MB/s]
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=16VIuHLgPGpX4J723Wd48UAPhHivLuUaH
From (redirected): https://drive.google.com/uc?id=16VIuHLgPGpX4J723Wd48UAPhHivLuUaH&confirm=t&uuid=ea1832a9-7377-465f-bcae-c220b3e50

In [3]:
eventos = pd.read_pickle('/content/CHEC/SuperEventos_Criticidad_AguasAbajo_CODEs.pkl')
eventos['FECHA']=pd.to_datetime(eventos['FECHA'])
redmt = pd.read_pickle('/content/CHEC/Data_CHEC/REDMT_1.pkl')
redmt['FECHA']=pd.to_datetime(redmt['FECHA'])
redmt['FECHA_C']=redmt['FECHA'].dt.to_period('M')
apoyos = pd.read_pickle('/content/CHEC/Data_CHEC/APOYOS.pkl')
apoyos['FECHA']=pd.to_datetime(apoyos['FECHA'])
apoyos['FECHA_C']=apoyos['FECHA'].dt.to_period('M')
switches = pd.read_pickle('/content/CHEC/Data_CHEC/SWITCHES.pkl')
switches['FECHA']=pd.to_datetime(switches['FECHA'])
switches['FECHA_C']=switches['FECHA'].dt.to_period('M')
trafos = pd.read_pickle('/content/CHEC/Data_CHEC/TRAFOS.pkl')
trafos['FECHA']=pd.to_datetime(trafos['FECHA'])
trafos['FECHA_C']=trafos['FECHA'].dt.to_period('M')

In [4]:
inicio = 0#math.ceil(len(eventos)/30)*24
final = 2#math.ceil(len(eventos)/30)*25
print(inicio,' - ', final)

0  -  2


In [5]:
# Ruta del archivo de salida
output_file = 'SuperEventos_Criticidad_AguasAbajo_'+str(inicio)+'-'+str(final)+'_.csv'

# Inicializa el archivo CSV si no existe
if not os.path.exists(output_file):
    columnas = [
                'evento',
                'equipo_ope',
                'tipo_equi_ope',
                'cto_equi_ope',
                'tipo_elemento',
                'inicio',
                'fin',
                'duracion_h',
                'tipo_duracion',
                'causa',
                'CNT_TRAFOS_AFEC',
                'cnt_usus',
                'SAIDI',
                'SAIFI',
                'ASSEMBLY',
                'PHASES',
                'FPARENT',
                'KV',
                'STATE',
                'LINESECTIO',
                'FECHA',
                'LONGITUD',
                'LATITUD',
                'DEP',
                'MUN',
                'OWNER1',
                'TRFTYPE',
                'ELNODE',
                'IMPEDANCE',
                'DATE_FAB',
                'MARCA',
                'TIPO_SUB',
                'GRUPO015',
                'FECHA_ACT',
                'KVA',
                'KV1',
                'ELNODE1',
                'ELNODE2',
                'CONDUCTOR',
                'NEUTRAL',
                'LENGTH',
                'KVNOM',
                'CLASS',
                'TOWNER',
                'MATERIALCONDUCTOR',
                'TIPOCONDUCTOR',
                'CALIBRECONDUCTOR',
                'GUARDACONDUCTOR',
                'NEUTROCONDUCTOR',
                'NEUTRO_SECUNDARIOCONDUCTOR',
                'NIVEL1CONDUCTOR',
                'NIVEL2CONDUCTOR',
                'NIVEL3CONDUCTOR',
                'NIVEL4CONDUCTOR',
                'ACOMETIDACONDUCTOR',
                'NEUTRO',
                'CALIBRENEUTRO',
                'CAPACITY',
                'RESISTANCE',
                'LONGITUD2',
                'LATITUD2',
                'NIVEL_C',
                'VALOR_C',
                'TRAMOS_AGUAS_ABAJO',
                'EQUIPOS_PUNTOS',
                'PUNTOS_POLIGONO'
               ]
    pd.DataFrame(columns=columnas).to_csv(output_file, index=False)

# DataFrame temporal para acumular las filas
temp_df = pd.DataFrame(columns=columnas)

In [6]:
# Inicializa variables
start_time = time.time()
start_time_1 = time.time()
error_eventos = 0
index_error = []

for index, row in eventos[inicio:final].iterrows():
    redmt_seleccionado = redmt.loc[(redmt['FECHA'].dt.year == row['FECHA'].year) &
                                (redmt['FECHA'].dt.month == row['FECHA'].month) &
                                (redmt['MUN'] == row['MUN'])]
    apoyos_seleccionado = apoyos.loc[(apoyos['FECHA'].dt.year == row['FECHA'].year) &
                                (apoyos['FECHA'].dt.month == row['FECHA'].month) &
                                (apoyos['MUN'] == row['MUN'])]
    switches_seleccionado = switches.loc[(switches['FECHA'].dt.year == row['FECHA'].year) &
                                (switches['FECHA'].dt.month == row['FECHA'].month) &
                                (switches['MUN'] == row['MUN'])]
    trafos_seleccionado = trafos.loc[(trafos['FECHA'].dt.year == row['FECHA'].year) &
                                (trafos['FECHA'].dt.month == row['FECHA'].month) &
                                (trafos['MUN'] == row['MUN'])]

    match row['tipo_equi_ope']:
        case 'interruptor':
            tramo_inicial = redmt.loc[(redmt['FECHA'].dt.year == row['FECHA'].year) &
                                    (redmt['FECHA'].dt.month == row['FECHA'].month) &
                                    (redmt['MUN'] == row['MUN']) & (redmt['CODE'] == row['LINESECTIO'])]

        case 'transformador':
            tramo_inicial = redmt.loc[(redmt['FECHA'].dt.year == row['FECHA'].year) &
                                    (redmt['FECHA'].dt.month == row['FECHA'].month) &
                                    ((redmt['ELNODE1'] == row['ELNODE']) | (redmt['ELNODE2'] == row['ELNODE']))]

        case _:
            tramo_inicial = redmt.loc[(redmt['FECHA'].dt.year == row['FECHA'].year) &
                                    (redmt['FECHA'].dt.month == row['FECHA'].month) &
                                    (redmt['MUN'] == row['MUN']) & (redmt['CODE'] == row['equipo_ope'])]

    try:
        if tramo_inicial['ORDER_'].values[0] == 1:
            nodo_inicial = (tramo_inicial['LATITUD'].values[0], tramo_inicial['LONGITUD'].values[0])
        else:
            nodo_inicial = (tramo_inicial['LATITUD2'].values[0], tramo_inicial['LONGITUD2'].values[0])

    except IndexError as e:
        print(f"Error al encontrar el nodo: {e}",
              f"Tramo: {row['LINESECTIO']}, en {row['MUN']} el {row['FECHA'].year}-{row['FECHA'].month}",
              f"Indice en tabla eventos: {index}")

        error_eventos += 1
        index_error.append(index)
        continue

    G = nx.DiGraph()

    for _, row_2 in redmt_seleccionado.iterrows():
        point1 = (row_2["LATITUD"], row_2["LONGITUD"])
        point2 = (row_2["LATITUD2"], row_2["LONGITUD2"])

        if row_2["ORDER_"] == 1:
            G.add_edge(point1, point2, id=row_2["CODE"])
        else:
            G.add_edge(point2, point1, id=row_2["CODE"])

    try:
        index_nodo_inicial = list(G.nodes).index(nodo_inicial)
    except ValueError as e:
        print(f"Error al encontrar el nodo: {e}",
              f"Tramo inicial: {tramo_inicial}",
              f"Red media tensión: {redmt_seleccionado}",
              f"Indice en tabla eventos: {index}")

        error_eventos += 1
        index_error.append(index)
        continue

    nodos_aguas_abajo = set(nx.descendants(G, nodo_inicial))
    nodos_aguas_abajo.add(nodo_inicial)

    if len(nodos_aguas_abajo) == 1:
        temp_df.loc[len(temp_df)] = [row['evento'],
                                 row['equipo_ope'],
                                 row['tipo_equi_ope'],
                                 row['cto_equi_ope'],
                                 row['tipo_elemento'],
                                 row['inicio'],
                                 row['fin'],
                                 row['duracion_h'],
                                 row['tipo_duracion'],
                                 row['causa'],
                                 row['CNT_TRAFOS_AFEC'],
                                 row['cnt_usus'],
                                 row['SAIDI'],
                                 row['SAIFI'],
                                 row['ASSEMBLY'],
                                 row['PHASES'],
                                 row['FPARENT'],
                                 row['KV'],
                                 row['STATE'],
                                 row['LINESECTIO'],
                                 row['FECHA'],
                                 row['LONGITUD'],
                                 row['LATITUD'],
                                 row['DEP'],
                                 row['MUN'],
                                 row['OWNER1'],
                                 row['TRFTYPE'],
                                 row['ELNODE'],
                                 row['IMPEDANCE'],
                                 row['DATE_FAB'],
                                 row['MARCA'],
                                 row['TIPO_SUB'],
                                 row['GRUPO015'],
                                 row['FECHA_ACT'],
                                 row['KVA'],
                                 row['KV1'],
                                 row['ELNODE1'],
                                 row['ELNODE2'],
                                 row['CONDUCTOR'],
                                 row['NEUTRAL'],
                                 row['LENGTH'],
                                 row['KVNOM'],
                                 row['CLASS'],
                                 row['TOWNER'],
                                 row['MATERIALCONDUCTOR'],
                                 row['TIPOCONDUCTOR'],
                                 row['CALIBRECONDUCTOR'],
                                 row['GUARDACONDUCTOR'],
                                 row['NEUTROCONDUCTOR'],
                                 row['NEUTRO_SECUNDARIOCONDUCTOR'],
                                 row['NIVEL1CONDUCTOR'],
                                 row['NIVEL2CONDUCTOR'],
                                 row['NIVEL3CONDUCTOR'],
                                 row['NIVEL4CONDUCTOR'],
                                 row['ACOMETIDACONDUCTOR'],
                                 row['NEUTRO'],
                                 row['CALIBRENEUTRO'],
                                 row['CAPACITY'],
                                 row['RESISTANCE'],
                                 row['LONGITUD2'],
                                 row['LATITUD2'],
                                 row['NIVEL_C'],
                                 row['VALOR_C'],
                                 list(nodos_aguas_abajo),
                                 list(nodos_aguas_abajo),
                                 0
                                ]
        continue

    # Detectar los puntos dispersos que están cerca de las conexiones aguas abajo
    # Crear geometrías para los puntos inicial y final
    gdf_redmt = gpd.GeoDataFrame(
        redmt_seleccionado,
        geometry=gpd.points_from_xy(redmt_seleccionado['LONGITUD'], redmt_seleccionado['LATITUD'])
    )

    # Añadir una columna de geometría para el segundo punto
    gdf_redmt['geometry2'] = gpd.points_from_xy(redmt_seleccionado['LONGITUD2'], redmt_seleccionado['LATITUD2'])

    # Filtrar líneas que están completamente en nodos_aguas_abajo
    gdf_redmt_filtrado = gdf_redmt[
        gdf_redmt.apply(lambda row:
            (row.geometry.y,row.geometry.x) in nodos_aguas_abajo and
            (row.geometry2.y,row.geometry2.x) in nodos_aguas_abajo,
            axis=1)]

    line_series = gdf_redmt_filtrado.apply(
        lambda row: LineString([
            (row.geometry.x, row.geometry.y),
            (row.geometry2.x, row.geometry2.y)
        ]),
        axis=1
    )

    gdf_redmt_filtrado = gdf_redmt_filtrado.copy()
    gdf_redmt_filtrado['line'] = line_series

    # Crear GeoDataFrames para otros elementos
    gdf_trafos = gpd.GeoDataFrame(
        trafos_seleccionado,
        geometry=gpd.points_from_xy(trafos_seleccionado['LONGITUD'], trafos_seleccionado['LATITUD'])
    )

    gdf_apoyos = gpd.GeoDataFrame(
        apoyos_seleccionado,
        geometry=gpd.points_from_xy(apoyos_seleccionado['LONGITUD'], apoyos_seleccionado['LATITUD'])
    )

    gdf_switches = gpd.GeoDataFrame(
        switches_seleccionado,
        geometry=gpd.points_from_xy(switches_seleccionado['LONGITUD'], switches_seleccionado['LATITUD'])
    )

    # Umbral de proximidad
    umbral = 0.0005

    # Configurar la geometría como líneas
    gdf_redmt_filtrado = gdf_redmt_filtrado.set_geometry('line')

    # TRAFOS
    # Calcular la distancia entre cada punto y las líneas
    gdf_trafos['distance_to_line'] = gdf_trafos.geometry.apply(
        lambda point: gdf_redmt_filtrado.distance(point).min()
    )

    # Filtrar por umbral de distancia
    trafos_cercanos = gdf_trafos[gdf_trafos['distance_to_line'] <= umbral]

    # Configurar la geometría de gdf_redmt_filtrado como líneas
    gdf_redmt_filtrado = gdf_redmt_filtrado.set_geometry('line')

    # SWITCHES
    # Calcular la distancia entre cada punto y las líneas
    gdf_switches['distance_to_line'] = gdf_switches.geometry.apply(
        lambda point: gdf_redmt_filtrado.distance(point).min()
    )

    # Filtrar por umbral de distancia
    switches_cercanos = gdf_switches[gdf_switches['distance_to_line'] <= umbral]

    # APOYOS
    # Calcular la distancia entre cada punto y las líneas
    gdf_apoyos['distance_to_line'] = gdf_apoyos.geometry.apply(
        lambda point: gdf_redmt_filtrado.distance(point).min()
    )

    # Filtrar por umbral de distancia
    apoyos_cercanos = gdf_apoyos[gdf_apoyos['distance_to_line'] <= umbral]

    # Combinar resultados
    puntos_cercanos = pd.concat([trafos_cercanos, apoyos_cercanos, switches_cercanos])

    # Calcular convex hull si hay más de 2 puntos
    if len(puntos_cercanos) > 2:
        # Crear GeoDataFrame de puntos cercanos
        gdf_puntos_cercanos = gpd.GeoDataFrame(
            puntos_cercanos,
            geometry=gpd.points_from_xy(puntos_cercanos['LONGITUD'], puntos_cercanos['LATITUD'])
        )
        try:
            # Calcular el convex hull
            hull = gdf_puntos_cercanos.unary_union.convex_hull

            if hull.geom_type == 'Polygon':
                # Caso normal: es un polígono
                polygon_puntos = list(hull.exterior.coords)
            elif hull.geom_type == 'LineString':
                # Caso colineal: crear una banda usando un buffer
                buffer_width = 0.001  # Define el ancho de la banda (ajústalo según tus necesidades)
                band = hull.buffer(buffer_width, cap_style=2)  # cap_style=2 para bordes rectos
                polygon_puntos = list(band.exterior.coords)
            else:
                # Caso inesperado
                print("Sin polígono")
                polygon_puntos = 0

        except Exception as e:
            # Manejo general de excepciones
            print("Sin polígono")
            polygon_puntos = 0
    else:
        polygon_puntos = 0

    transformed_points = (
        list(map(lambda t: (t[1], t[0]), polygon_puntos))
        if not isinstance(polygon_puntos, int)
        else polygon_puntos
        )

    # Crear la lista con las coordenadas left
    puntos_cercanos = list(zip(puntos_cercanos['LATITUD'], puntos_cercanos['LONGITUD']))

    temp_df.loc[len(temp_df)] = [row['evento'],
                                 row['equipo_ope'],
                                 row['tipo_equi_ope'],
                                 row['cto_equi_ope'],
                                 row['tipo_elemento'],
                                 row['inicio'],
                                 row['fin'],
                                 row['duracion_h'],
                                 row['tipo_duracion'],
                                 row['causa'],
                                 row['CNT_TRAFOS_AFEC'],
                                 row['cnt_usus'],
                                 row['SAIDI'],
                                 row['SAIFI'],
                                 row['ASSEMBLY'],
                                 row['PHASES'],
                                 row['FPARENT'],
                                 row['KV'],
                                 row['STATE'],
                                 row['LINESECTIO'],
                                 row['FECHA'],
                                 row['LONGITUD'],
                                 row['LATITUD'],
                                 row['DEP'],
                                 row['MUN'],
                                 row['OWNER1'],
                                 row['TRFTYPE'],
                                 row['ELNODE'],
                                 row['IMPEDANCE'],
                                 row['DATE_FAB'],
                                 row['MARCA'],
                                 row['TIPO_SUB'],
                                 row['GRUPO015'],
                                 row['FECHA_ACT'],
                                 row['KVA'],
                                 row['KV1'],
                                 row['ELNODE1'],
                                 row['ELNODE2'],
                                 row['CONDUCTOR'],
                                 row['NEUTRAL'],
                                 row['LENGTH'],
                                 row['KVNOM'],
                                 row['CLASS'],
                                 row['TOWNER'],
                                 row['MATERIALCONDUCTOR'],
                                 row['TIPOCONDUCTOR'],
                                 row['CALIBRECONDUCTOR'],
                                 row['GUARDACONDUCTOR'],
                                 row['NEUTROCONDUCTOR'],
                                 row['NEUTRO_SECUNDARIOCONDUCTOR'],
                                 row['NIVEL1CONDUCTOR'],
                                 row['NIVEL2CONDUCTOR'],
                                 row['NIVEL3CONDUCTOR'],
                                 row['NIVEL4CONDUCTOR'],
                                 row['ACOMETIDACONDUCTOR'],
                                 row['NEUTRO'],
                                 row['CALIBRENEUTRO'],
                                 row['CAPACITY'],
                                 row['RESISTANCE'],
                                 row['LONGITUD2'],
                                 row['LATITUD2'],
                                 row['NIVEL_C'],
                                 row['VALOR_C'],
                                 str(nodos_aguas_abajo),
                                 puntos_cercanos,
                                 transformed_points
                                ]

    # Cada 10 iteraciones, guarda en el archivo CSV
    if (index + 1) % 1000 == 0:
        temp_df.to_csv(output_file, mode='a', header=False, index=False)
        # Reinicia el DataFrame temporal
        temp_df = pd.DataFrame(columns=columnas)
    # Verifica si han pasado 5 minutos
    elapsed_time = time.time() - start_time
    if elapsed_time >= 600:
        end_time = time.time()
        elapsed_time = end_time - start_time_1

        # Convertir el tiempo transcurrido a minutos
        elapsed_time_minutes = elapsed_time / 60
        print(f"Evento {index+1}, tiempo transcurrido: {elapsed_time_minutes:.2f} minutos")
        start_time = time.time()  # Resetea el tiempo de inicio

# Guarda cualquier dato restante al final
if len(temp_df) > 0:
    temp_df.to_csv(output_file, mode='a', header=False, index=False)
    print("Guardadas filas restantes")

end_time = time.time()
elapsed_time = end_time - start_time_1
# Convertir el tiempo transcurrido a minutos
elapsed_time_minutes = elapsed_time / 60

Guardadas filas restantes


In [7]:
df = pd.read_csv(output_file, nrows=10)
df['inicio']=pd.to_datetime(df['inicio'])
df['FECHA_C'] = df['inicio'].dt.to_period('M')


In [8]:
df

,evento,equipo_ope,tipo_equi_ope,cto_equi_ope,tipo_elemento,inicio,fin,duracion_h,tipo_duracion,causa,...,CAPACITY,RESISTANCE,LONGITUD2,LATITUD2,NIVEL_C,VALOR_C,TRAMOS_AGUAS_ABAJO,EQUIPOS_PUNTOS,PUNTOS_POLIGONO,FECHA_C
0,7549809,MAZ30L13,interruptor,MAZ30L13,33,2019-01-01 00:00:01,2019-01-01 01:01:49,1.030000,> 3 min,Falla en postes y/o crucetas en el SDL,...,NaN,NaN,NaN,NaN,0,0.0,"{(5.270261453975187, -75.13586881335723), (5.2...","[(5.279440126576255, -75.09663447048119), (5.2...","[(5.26300304370033, -75.14657420539724), (5.26...",2019-01
1,7549809,MAZ30L13,interruptor,MAZ30L13,33,2019-01-01 00:00:01,2019-01-01 01:09:36,1.159722,> 3 min,Falla en postes y/o crucetas en el SDL,...,NaN,NaN,NaN,NaN,3,0.0,"{(5.270261453975187, -75.13586881335723), (5.2...","[(5.279440126576255, -75.09663447048119), (5.2...","[(5.26300304370033, -75.14657420539724), (5.26...",2019-01
